In [ ]:
import pandas as pd
import numpy as np
import os

path = os.path.join(os.getcwd(), r"Redacted-Department _ CAD Reconciliation.csv")

df = pd.read_csv(path)

# a1. Data range
#print("Earliest dispatch:", df['Dispatch_Date'].min())
#print("Latest dispatch:", df['Dispatch_Date'].max())
# a2. % of missing data
missing_percent = df.isna().mean() * 100
print("Missing data % by column:\n", missing_percent)
# a3.

# a4.
df_clean = df.dropna()

# b.1
df_clean["ALARM DATE TIME"] = pd.to_datetime(df_clean["ALARM DATE TIME"], errors="coerce")
df_clean["CALL COMPLETE"] = pd.to_datetime(df_clean["CALL COMPLETE"], errors="coerce")

df_clean = df_clean.dropna(subset=["ALARM DATE TIME", "CALL COMPLETE"])

df_clean["Resolution_Time"] = (df_clean["CALL COMPLETE"] - df_clean["ALARM DATE TIME"]).dt.total_seconds() / 60

avg_resolution = df_clean["Resolution_Time"].mean()
print(f"Average resolution time: {avg_resolution:.2f} minutes")

# b.2
df_clean["Num_Units"] = df_clean["DISPATCH UNIT"].apply(lambda x: len(str(x).split(",")))
avg_units = df_clean["Num_Units"].mean()
print(f"Average number of fire units per alarm: {avg_units:.2f}")

# b.3
shift_counts = df_clean["SHIFT"].value_counts()
print("Number of alarms per shift:\n", shift_counts)

busiest_shift = shift_counts.idxmax()
print(f"\nThe busiest shift is: {busiest_shift}")

# b.4

Missing data % by column:
 XREF ID                   0.000000
DISPATCH UNIT             0.000000
DISPATCH CREATED DATE     0.000000
INCIDENT NUMBER           0.000000
1ST UNIT ON SCENE        19.454545
ALARM DATE TIME           1.409091
CALL COMPLETE             1.409091
SHIFT                     3.136364
dtype: float64
Average resolution time: 125722.06 minutes
Average number of fire units per alarm: 1.48


C:\Users\ataba\AppData\Local\Temp\ipykernel_30068\792216643.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean["ALARM DATE TIME"] = pd.to_datetime(df_clean["ALARM DATE TIME"], errors="coerce")
C:\Users\ataba\AppData\Local\Temp\ipykernel_30068\792216643.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["ALARM DATE TIME"] = pd.to_datetime(df_clean["ALARM DATE TIME"], errors="coerce")
C:\Users\ataba\AppData\Local\Temp\ipykernel_30068\792216643.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent